## IR Assignment Group-082

## Group members

<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>

<table>
  <tr>
    <th>Full Name</th>
    <th>BITS ID</th>
  </tr>
  <tr>
    <td>T V Sriharini</td>
    <td>2020fc04020@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Subodh Kant Mishra</td>
    <td>2020fc04064@wilp.bits-pilani.ac.in</td>
  </tr>
  <tr>
    <td>Mriganka Shekhar Gayen</td>
    <td>2020fc04069@wilp.bits-pilani.ac.in</td>
  </tr>
</table>

Given a Dataset collected from the Centre for Inventions and Scientific Information ("CISI") and consists of text data about 1,460 documents and 112 associated queries. Its purpose is to be used to build models of information retrieval where a given query will return a list of document IDs relevant to the query. The file "CISI.REL" contains the correct list (ie. "gold standard" or "ground proof") of query-document matching and your model can be compared against this "gold standard" to see how it has performed.

### 1. Load the data set and import necessary libraries (1 Marks)
We import various packages used for text and general data processing
* Pandas, Numpy, Skleaarn - For Data Processing
* NLTK - Natural Language Tool Kit for Natural Language Processing
* OS, functools - General Packages for Processing files


In [2]:
## Library imports
import numpy as np 
import pandas as pd
from rank_bm25 import BM25Okapi

from sklearn.feature_extraction.text import CountVectorizer
#we will use Scikit-Learn's TfidfVectorizer, which operates similarly to CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#implementation using scikit-learn
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

import os, glob, re, sys, random, unicodedata, collections,math
from tqdm import tqdm
from functools import reduce
import nltk

from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kants\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kants\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1.1 Importing the files - All FIle

We are importing the CISI.ALL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [3]:
def extract_text(path,filetype="ALL"):
    """
    Extracts text from a given file path.
    """

    qry_id=0
    doc_id=0
    doc_text = ""
    rel_set = {}
    doc_set = {}
    qry_set = {}
    lines = ""
    with open(path, 'r', encoding='utf-8') as f:
        if filetype == "ALL" or filetype == "QRY":
            for l in f.readlines():
                lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
            lines = lines.lstrip("\n").split("\n")
        elif filetype == "REL":
            for l in f.readlines():
                qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
                doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
                if qry_id in rel_set:
                    rel_set[qry_id].append(doc_id)
                else:
                    rel_set[qry_id] = []
                    rel_set[qry_id].append(doc_id)
            return rel_set
        else:
            print("Invalid file type")
            return lines
    if filetype == "ALL": 
        for l in lines:
            if l.startswith(".I"):
                doc_id = int(l.split(" ")[1].strip())-1
            elif l.startswith(".X"):
                doc_set[doc_id] = doc_text.lstrip(" ")
                doc_id = ""
                doc_text = ""
            else:
                doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.
        return doc_set
    elif filetype == "QRY":
        qry_id = ""
        for l in lines:
            if l.startswith(".I"):
                qry_id = int(l.split(" ")[1].strip()) -1
            elif l.startswith(".W"):
                qry_set[qry_id] = l.strip()[3:]
                qry_id = ""
        return qry_set

In [4]:
### Processing DOCUMENTS
BASE_FOLDER = f"{os.getcwd()}\\archive\\"
doc_set = extract_text(f'{BASE_FOLDER}CISI.ALL',filetype="ALL")
print(f'Read {len(doc_set)} documents')

Read 1460 documents


### 1.2 Importing the files - Queries FIle

We are importing the CISI.QRY file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [5]:
qry_set = extract_text(f'{BASE_FOLDER}CISI.QRY',filetype="QRY")
print(f'Read {len(qry_set)} documents')

Read 112 documents


### 1.3 Importing the files - Rel FIle

We are importing the CISI.QREL file where we perform text processing such as removing newline characters, and '.' characters,
and split the data into different variables.

In [6]:
rel_set = extract_text(f'{BASE_FOLDER}CISI.REL',filetype="REL")
print(f'Read {len(rel_set)} documents')

Read 76 documents


### 2.1 Extract features from text files. Each unique word in the document can be considered as a feature (1 Mark)

We check the General statistics of the file provided

In [7]:
## Here we check some statistics and info of CISI dataset

print(f'Read {len(doc_set)} documents, {len(qry_set)} queries and {len(rel_set)} mappings from CISI dataset')

number_of_rel_docs = [len(value) for key, value in rel_set.items()]
print('Average %.2f and %d min number of relevant documents by query ' % 
      (np.mean(number_of_rel_docs), np.min(number_of_rel_docs)))

print('Queries without relevant documents: ', 
      np.setdiff1d(list(qry_set.keys()),list(rel_set.keys())))

Read 1460 documents, 112 queries and 76 mappings from CISI dataset
Average 40.97 and 1 min number of relevant documents by query 
Queries without relevant documents:  [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


#### Checking the sample of relevant document from query and its content

In [8]:
random.seed(42)
idx = random.sample(rel_set.keys(),1)[0]

print('Query ID %s ==>' % idx, qry_set[idx])
rel_docs = rel_set[idx]
print('Documents relevants to Query ID %s' % idx, rel_docs)
sample_document_idx = random.sample(rel_docs,1)[0]
print('Document ID %s ==>' % sample_document_idx, doc_set[sample_document_idx])

Query ID 14 ==> How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
Documents relevants to Query ID 14 [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
Document ID 48 ==> Adaptive Information Dissemination Sage, C.R. Anderson, R.R. Fitzwater, D.R. Computer dissemination of information offers significant advantages over manual dissemination because the computer can use strategies that are impractical and in some cases impossible for a human.. This paper describes the Ames Laboratory Selective Dissemination of Information syste

## Deriving Feature using Count Vectorizer Method

In [9]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
    
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

# put the common code into several methods
def get_keywords(idx,docs_test,cv,feature_names):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords,docs_body):
    # now print the results
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data
    
def calculate_similarity(X, vectorizor, query, top_k=10):
    """ Vectorizes the `query` via `vectorizor` and calculates the cosine similarity of 
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""
    
    # Vectorize the query to the same length as documents
    query_vec = vectorizor.transform(query)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)


#### Preprocess The Dataset

In [10]:
data = pd.DataFrame.from_dict(doc_set, orient='index',columns=['text'])
data['process_text'] = data['text'].apply(lambda x:preprocess(x))
data['unique_word'] = data['text'].apply(lambda x:set(x.split()))
data

,text,process_text,unique_word
0,18 Editions of the Dewey Decimal Classificatio...,eighteen edit dewey decim classif comaromi pr...,"{will, his, attempt, 1971,, have, describe, Cl..."
1,"Use Made of Technical Libraries Slater, M. Thi...",use made technic librari slater report analys...,"{Taking, technology, is, with, Kingdom., use.,..."
2,Two Kinds of Power An Essay on Bibliographic C...,two kind power essay bibliograph control wils...,"{storehouses, will, As, written, familiar, Two..."
3,Systems Analysis of a University Library; fina...,system analysi univ librari final report rese...,"{Repot, provoked, new, Libraries, Analysis, at..."
4,A Library Management Game: a report on a resea...,librari manag game report research project br...,"{fields, professional, courses, main, times,, ..."
...,...,...,...
1455,"World Dynamics Forrester, J.W. Over the last s...",world dynam forrest last sever decad interest...,"{economic, believe, most, world-wide, last, fo..."
1456,"World Trends in Library Education Bramley, G. ...",world trend librari educ bramley one signif a...,"{new, professional, implications, had, while, ..."
1457,Legal Restrictions on Exploitation of the Pate...,legal restrict exploit patent monopoli econom...,"{remarkable, encouragement, making,, Analysis,..."
1458,"Language and Thought Poluskin, V.A. This book ...",languag thought poluskin book consid basic as...,"{problem, Language, thought,, abstraction,, re..."


#### Using scikit Learn Count Vectorizer Method

In [11]:
# count term frequency using CountVectorizer from scikit-learn
cv = CountVectorizer(stop_words=set(stopwords.words('english'))) 
X = cv.fit_transform(data['process_text'])
print('Non Zeros doc matrix',cv.transform(data['text']).count_nonzero())
df = pd.DataFrame(X.todense(), columns=cv.get_feature_names())
print('First 10 Vocabulary from document set: ',list(cv.vocabulary_.keys())[:10])
df
#word_tokenize(str(preprocess(doc_set[0])))

Non Zeros doc matrix 28708
First 10 Vocabulary from document set:  ['eighteen', 'edit', 'dewey', 'decim', 'classif', 'comaromi', 'present', 'studi', 'histori', 'first']


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,12k,14th,15th,1961b,1967a,1971and,24th,2d,2x2,2x8,...,zimmerman,zipf,zipfian,zipper,zone,zoolog,zuckerman,zund,zvezhinskii,zyabrev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1458,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Using ScikitLear TfidfVectorizer

In [12]:
tfidf_vectorizor = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word', 
                                       stop_words='english', ngram_range=(1, 1), 
                                       norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,)
X_tf = tfidf_vectorizor.fit_transform(data['process_text'])
features = tfidf_vectorizor.get_feature_names()
df_tf = pd.DataFrame(X_tf.todense(), columns=features)
print('First 10 Vocabulary from document set: ',list(tfidf_vectorizor.vocabulary_.keys())[:10])
print('tfidf matrix successfully created.','\nLength of features',len(features))

df_tf

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


First 10 Vocabulary from document set:  ['eighteen', 'edit', 'dewey', 'decim', 'classif', 'comaromi', 'present', 'studi', 'histori', 'ddc']
tfidf matrix successfully created. 
Length of features 6646


,12k,14th,15th,1961b,1967a,1971and,24th,2d,2x2,2x8,...,zimmerman,zipf,zipfian,zipper,zone,zoolog,zuckerman,zund,zvezhinskii,zyabrev
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4. Implement pseudo-relevance feedback.(1Mark)

In [13]:
idx = 20
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

# Index all documents using BM25
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Process query and get scores for each indexed document using BM25
tokenized_query = query.split(" ")
print('Query ==> ', query, '\nRelevant documents IDs: ==> ', rel_docs, '\n',len(rel_docs),' relevant documents')
scores = bm25.get_scores(tokenized_query)
print('bm25 scores -----> ',scores,'\nLength of scores',len(scores))
doc_set[scores.argmax()]

Query ==>  The need to provide personnel for the information field. 
Relevant documents IDs: ==>  [5, 13, 21, 84, 170, 184, 185, 302, 338, 391, 399, 452, 838, 890, 895, 922, 923, 948, 984, 1019, 1020, 1146, 1148, 1205, 1355] 
 25  relevant documents
bm25 scores ----->  [10.96997353  7.75949152  8.81442548 ...  9.52562468  5.60350587
  8.18047688] 
Length of scores 1460


"The Hidden Dimension Hall, E.T. Generally speaking, there are two types of books of interest to the serious reader today:  those that are content oriented, designed to convey a particular body of knowledge, and those that deal with structure, the way in which events are organized.  It is doubtful if an author has any control over which of these two types of books he writes, though it is desirable that he be aware of the difference.  The same applies to the reader whose satisfaction depends largely on his unstated expectations.  In today's world, when all of us are overwhelmed with data from many sources, it is easy to understand why people are apt to feel that they are losing touch with developments even in their own field.  One senses that there is also a growing awareness of a loss of relatedness to the world at large.  This loss of relatedness leads to an increased need for organizing frames of reference to aid in intergrating the mass of rapidly changing information with which man

In [14]:
##Source: https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

mean_reciprocal_rank([[0, 0, 1], [0, 1, 0], [1, 0, 0]])


0.611111111111111

In [15]:
## Argsort gives the indexes of values in increasing order, so we input with the negative values of scores
most_relevant_documents = np.argsort(-scores)

print(most_relevant_documents[:100]) # printing first 100 most relevant results

## Mask relevant documents with 0's and 1's according to query <-> document annotation
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

print(sorted_masked_relevance_results[:100]) #printing first 100 results: 1 is relevant 0 isn't

# Calculate MRR@10
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

[ 411  895  584  340  922  113  691  546  947  474 1069 1259 1223  552
  132  949  827  247 1409 1204  720 1093  897  141  593  338 1098  597
  647   26 1357  124  975  230  670  643  155  184  197  359  284  387
  274  492  582  162 1188 1324  382  187 1406 1398  383  403 1439  233
  407  240  266  842  366  718  444   16  258  409  127 1429 1361 1156
  846  589  123  239    8  464  946 1244  259  882  251 1445  957  631
  977  581  938  659 1317  306  511 1356 1413  215    7  480 1351  629
  458 1359]
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
0.5


### 3. Compute TF and TF-IDF factors (1Mark)
We use the tfidf transformer to compute the tf and IDF Factors for the dataset
##### TF-IDF Calculation using TFIDF Transformer and tfidfVectorizer

In [16]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X_tf)
#data = pd.DataFrame.from_dict(doc_set, orient='index',columns=['text'])
feature_names=tfidf_vectorizor.get_feature_names()

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
#generate tf-idf for all documents in your list. docs_test has 1460 documents
tf_idf_vector=tfidf_transformer.transform(tfidf_vectorizor.transform(doc_set.values()))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,20)
    
    
    results.append(keywords)

data['keywords']=results
data

,text,process_text,unique_word,keywords
0,18 Editions of the Dewey Decimal Classificatio...,eighteen edit dewey decim classif comaromi pr...,"{will, his, attempt, 1971,, have, describe, Cl...","{'ddc': 0.467, 'dewey': 0.438, 'eighteenth': 0..."
1,"Use Made of Technical Libraries Slater, M. Thi...",use made technic librari slater report analys...,"{Taking, technology, is, with, Kingdom., use.,...","{'slater': 0.355, 'desk': 0.355, 'transfer': 0..."
2,Two Kinds of Power An Essay on Bibliographic C...,two kind power essay bibliograph control wils...,"{storehouses, will, As, written, familiar, Two...","{'power': 0.469, 'slogan': 0.417, 'wilson': 0...."
3,Systems Analysis of a University Library; fina...,system analysi univ librari final report rese...,"{Repot, provoked, new, Libraries, Analysis, at...","{'ugc': 0.533, 'repot': 0.533, 'buckland': 0.4..."
4,A Library Management Game: a report on a resea...,librari manag game report research project br...,"{fields, professional, courses, main, times,, ...","{'game': 0.669, 'women': 0.301, 'spread': 0.24..."
...,...,...,...,...
1455,"World Dynamics Forrester, J.W. Over the last s...",world dynam forrest last sever decad interest...,"{economic, believe, most, world-wide, last, fo...","{'world': 0.425, 'interplay': 0.403, 'mutual':..."
1456,"World Trends in Library Education Bramley, G. ...",world trend librari educ bramley one signif a...,"{new, professional, implications, had, while, ...","{'bramley': 0.455, 'potent': 0.408, 'twentieth..."
1457,Legal Restrictions on Exploitation of the Pate...,legal restrict exploit patent monopoli econom...,"{remarkable, encouragement, making,, Analysis,...","{'legal': 0.409, 'morn': 0.389, 'baxter': 0.38..."
1458,"Language and Thought Poluskin, V.A. This book ...",languag thought poluskin book consid basic as...,"{problem, Language, thought,, abstraction,, re...","{'poluskin': 0.697, 'thought': 0.614, 'complex..."


### 5. Retrieve the top k- documents using TF-IDF model(2Marks)

###### Calculating Cosine Similarity

In [18]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

print(cosine_similarity(tf_idf_vector),'\ncosine similarity for first document\n',cosine_similarity(tf_idf_vector)[0])

[[1.         0.         0.         ... 0.         0.         0.00453827]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.07362287 0.         0.        ]
 ...
 [0.         0.         0.07362287 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.00453827 0.         0.         ... 0.         0.         1.        ]] 
cosine similarity for first document
 [1.         0.         0.         ... 0.         0.         0.00453827]


#### Calculating Top K(10) Document based on Cosine Similarity for each query

In [19]:
for key in qry_set.keys():

    user_question = []
    user_question.append(qry_set[key])
    print(user_question)
    sim_vecs, cosine_similarities = calculate_similarity(X_tf, tfidf_vectorizor, user_question)
    print('Similar Doc Id',sim_vecs,'Cosine Similarity',[cosine_similarities[i] for i in sim_vecs])
    #print(rel_set[key],'\n----------------------')
    #show_similar_documents(data, cosine_similarities, sim_vecs)

['What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?']
Similar Doc Id [1029 1018   70  446  905  872  804   34  223  743] Cosine Similarity [0.1809991965128494, 0.16793493893911618, 0.14740139478799813, 0.14663589124571264, 0.1458362426448593, 0.1449649543237236, 0.14189811351215142, 0.1398631236465585, 0.1398317961298713, 0.13771136591536717]
['How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?']
Similar Doc Id [1375 1095  992 1135  126  566  821  373 1376  124] Cosine Similarity [0.287490424302825, 0.23257645111886516, 0.21762007735944966, 0.21732278845991515, 0.21081522188256083, 0.20386203673160094, 0.19490563054561325, 0.1945901474525878, 0.19369750450088435, 0.18912026779332805]
['What is informatio

### 6. Identify terms in these documents which are distinctive to the documents.(2Marks)

In [20]:
idx=10
print(f'Content of document{idx}\n',data['text'][idx],f'\nTop 20 keyword of document {idx}\n',data['keywords'][idx],'\nUnique words in document\n',data['unique_word'][idx])

Content of document10
 The Acquisition of Library Materials Ford, S. The scope of acquisitions work, outlined in the Introduction, acknowledges the importance of selection policy, serials recording, and other topics kindred to acquisitions. These topics are discussed in this book only as they relate to obtaining library materials.  They are examined thoroughly in books and papers that are cited in the references and the bibliographic note. Centralized acquisitions and automation of order routines are of major importance in order work and they are reviewed as chapters in this book.  These chapters are introductions to the concepts and problems of centralization and automation, not manuals of practice. For treatment of these topics in particular and in depth the reader is referred to the references cited.  For automation these references are only a modest selection from an enormous literature.  
Top 20 keyword of document 10
 {'ford': 0.518, 'modest': 0.426, 'depth': 0.338, 'treatment': 

#### Getting Result from Query and showing the Mean Reciprocal Rank to overall queries 

In [27]:
def results_from_query(qry_id,qry_set,bm25,process_flag=False,distinct_flag=False,distinct_terms=""):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        qry_id (int): id of query on dataset
        bm25 (object): indexed corpus

    Returns:
        boolean sorted relevance array of documents
    """    
    query = qry_set[qry_id]
    rel_docs = []
    if distinct_flag:
        query = query + distinct_terms
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    if process_flag:
        tokenized_query = preprocess_string(query, True, True, True)
    else:
        tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-scores)
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results

In [28]:
stemmer = nltk.stem.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')
def preprocess_string(txt, remove_stop=True, do_stem=True, to_lower=True):
    """
    Return a preprocessed tokenized text.
    
    Args:
        txt (str): original text to process
        remove_stop (boolean): to remove or not stop words (common words)
        do_stem (boolean): to do or not stemming (suffixes and prefixes removal)
        to_lower (boolean): remove or not capital letters.
        
    Returns:
        Return a preprocessed tokenized text.
    """    
    if to_lower:
        txt = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt)
    
    if remove_stop:
        tokens = [tk for tk in tokens if tk not in stop_words]
    if do_stem:
        tokens = [stemmer.stem(tk) for tk in tokens]
    return tokens

In [29]:
corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(corpus)
results = [results_from_query(qry_id, qry_set,bm25) for qry_id in list(qry_set.keys())]
print('Without Tokenization: MRR@10 %.4f' % mean_reciprocal_rank(results))

corpus = list(doc_set.values())
tokenized_corpus = [preprocess_string(doc, remove_stop=True, do_stem=True, to_lower=True) for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)
results = [results_from_query(qry_id, qry_set,bm25,process_flag=True) for qry_id in list(qry_set.keys())]
print('With Tokenization MRR@10 %.4f' % mean_reciprocal_rank(results))

Without Tokenization: MRR@10 0.3146
With Tokenization MRR@10 0.4187


### 7. Add the terms to the query, and re-run the retrieval process. Return the final results.(2Marks)

In [30]:
idx= 10
for keys in data['keywords'][idx]:
    print(f'Adding {keys} to the query')
    results = [results_from_query(qry_id, qry_set,bm25,process_flag=True,distinct_flag=True,distinct_terms=keys) for qry_id in list(qry_set.keys())]
    print('With Tokenization and disinct term MRR@10 %.4f \n' % mean_reciprocal_rank(results))

Adding ford to the query
With Tokenization and disinct term MRR@10 0.4238 

Adding modest to the query
With Tokenization and disinct term MRR@10 0.4257 

Adding depth to the query
With Tokenization and disinct term MRR@10 0.4257 

Adding treatment to the query
With Tokenization and disinct term MRR@10 0.4222 

Adding scope to the query
With Tokenization and disinct term MRR@10 0.4168 

Adding note to the query
With Tokenization and disinct term MRR@10 0.4236 

Adding order to the query
With Tokenization and disinct term MRR@10 0.4098 

Adding reader to the query
With Tokenization and disinct term MRR@10 0.4259 

Adding work to the query
With Tokenization and disinct term MRR@10 0.4133 

Adding particular to the query
With Tokenization and disinct term MRR@10 0.4188 

Adding book to the query
With Tokenization and disinct term MRR@10 0.4257 

Adding major to the query
With Tokenization and disinct term MRR@10 0.4135 



In [ ]:
WORD_MIN_LENGTH = 2 ## we'll drop all tokens with less than this size

def strip_accents(text):
    """Strip accents and punctuation from text. 
    For instance: strip_accents("João e Maria, não entrem!") 
    will return "Joao e Maria  nao entrem "

    Parameters:
    text (str): Input text

    Returns:
    str: text without accents and punctuation

   """    
    nfkd = unicodedata.normalize('NFKD', text)
    newText = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\']', ' ', newText)

def tokenize_text(text):
    """Make all necessary preprocessing of text: strip accents and punctuation, 
    remove \n, tokenize our text, convert to lower case, remove stop words and 
    words with less than 2 chars.

    Parameters:
    text (str): Input text

    Returns:
    str: cleaned tokenized text

   """        
    text = strip_accents(text)
    text = re.sub(re.compile('\n'),' ',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS and len(word) >= WORD_MIN_LENGTH]
    return words
    
def inverted_index(words):
    """Create a inverted index of words (tokens or terms) from a list of terms

    Parameters:
    words (list of str): tokenized document text

    Returns:
    Inverted index of document (dict)

   """       
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    """Insert document id into Inverted Index

    Parameters:
    inverted (dict): Inverted Index
    doc_id (int): Id of document been added
    doc_index (dict): Inverted Index of a specific document.

    Returns:
    Inverted index of document (dict)

   """        
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

## Using AND as logical operator
def boolean_search(inverted, file_names, query):
    """Run a boolean search with AND operator between terms over 
    the inverted index.

    Parameters:
    inverted (dict): Inverted Index
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Names of books that matchs the query.

   """      
    # preprocess the user query using same function used to build Inverted Index
    words = [word for _, word in enumerate(tokenize_text(query)) if word in inverted]
    # list with a disctinct document match for each term from query
    results = [set(inverted[word].keys()) for word in words]
    # AND operator. Replace & for | to modify to OR behavior.
    docs = reduce(lambda x, y: x & y, results) if results else []
    return ([file_names[doc] for doc in docs])

def ranked_search(k, tf_idf_index, file_names, query,tf_idf):
    """Run ranked query search using tf-idf model.

    Parameters:
    k (int): number of results to return
    tf_idf_index (dict): Data Structure storing Tf-Idf weights to each 
                        pair of (term,doc_id) 
    file_names (list): List with names of files (books)
    query (txt): Query text

    Returns:
    Top-k names of books that matchs the query.

   """   
    tokens = tokenize_text(query)
    query_weights = {}
    for doc_id, token in tf_idf:
        if token in tokens:
            query_weights[doc_id] = query_weights.get(doc_id, 0) + tf_idf_index[doc_id, token]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    results = []
    for i in query_weights[:k]:
        results.append(int(file_names[i[0]]))
    results.sort()
    return results